## Data Scraping

In [1]:
import urllib
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import string as st
import seaborn as sns

In [4]:
def top_2000_companies():
    
    r = urllib.request.urlopen('https://h1bdata.info/topcompanies.php') 
    soup = BeautifulSoup(r)

    data2 = soup.find_all('tr')    
    labels = []
    #get headings
    for h in data2[0].find_all('th'):
        labels.append(h.get_text().strip().lower())   
    final = []
    #get data
    for data in data2[1:]:
        data_list = []
        for d in data.find_all('td'):
            d_str = d.get_text().replace(',','')
            
            if d_str.isnumeric():
                data_list.append(int(d_str))
            else:
                data_list.append(d_str)                      
        final.append(data_list)
    
    df = pd.DataFrame(final, columns = labels) 
    df =df.drop(columns='latest filings')
    df = df.rename(columns={"#": "index", "company name": "company","# of h-1b filings": "total_filings","average salary":"avg_salary"})
    df = df.set_index('index')
    return df

In [5]:
df_companies = top_2000_companies()
df_companies = df_companies.dropna(how='any')
df_companies.to_csv('data/top2kcompanynames.csv')
df_companies

,company,total_filings,avg_salary
index,,,
1,INFOSYS LIMITED,117594.0,$83498
2,TATA CONSULTANCY SERVICES LIMITED,91518.0,$71575
3,COGNIZANT TECHNOLOGY SOLUTIONS US CORP,80977.0,$87624
4,DELOITTE CONSULTING LLP,58320.0,$93334
5,CAPGEMINI AMERICA INC,51256.0,$85797
...,...,...,...
1996,RISK MANAGEMENT SOLUTIONS INC,275.0,$116647
1997,INCANDESCENT TECHNOLOGIES INC,275.0,$85588
1998,DATAQUAD INC,275.0,$89346


In [7]:
def load_data(company,year):
    company = company.replace(' ','+')
    #print('https://h1bdata.info/index.php?em='+company+'&job=&city=&year='+str(year))
    #print(company)
    r = urllib.request.urlopen('https://h1bdata.info/index.php?em='+company+'&job=&city=&year='+str(year)) 
    soup = BeautifulSoup(r)

    data2 = soup.find_all('tr')    
#     labels = []
#     for h in data2[0].find_all('th'):
#         labels.append(h.get_text().strip().lower())

    final = []
    for data in data2[1:]:
        data_list = []
        for d in data.find_all('td'):
            d_str = d.get_text().replace(',','')

            if d_str.isnumeric():
                data_list.append(int(d_str))
            else:
                data_list.append(d_str)                      
        final.append(data_list)
    
    df = pd.DataFrame(final)    
    return df

In [111]:
#data extraction - first 100 companies , step is 10 to keep file size <100 mb
step =10
for i in range(0,100,step):
    frames = [load_data(company,year) for company in df_companies['company'].dropna().to_list()[i:i+step] for year in range(2012,2022) ]  
    result = pd.concat(frames)
    result.to_csv(f"data/{i}-{i+step}.csv")
    print("done "+str(i+step)+" companies")

done 10 companies
done 20 companies
done 30 companies
done 40 companies
done 50 companies
done 60 companies
done 70 companies
done 80 companies
done 90 companies
done 100 companies


In [9]:
#data extraction - don't run everytime
step =50
for i in range(900,1000,step):
    frames = [load_data(company,year) for company in df_companies['company'].dropna().to_list()[i:i+step] for year in range(2012,2022) ]  
    result = pd.concat(frames)
    result.to_csv(f"data/{i}-{i+step}.csv")
    print("done "+str(i+step)+" companies")

done 950 companies
done 1000 companies


## Data Loading

In [16]:
import glob
all_files = glob.glob('data/scraped_data_in_parts.csv')
result = pd.concat((pd.read_csv(f,index_col=0) for f in all_files))

ValueError: No objects to concatenate

In [12]:
result.columns = ['employer','job_title','base_salary','location','submit_date','start_date']
result['submit_date'] = pd.to_datetime(result['submit_date'])
result['start_date'] = pd.to_datetime(result['start_date'])
#result['state'] = result['location'].str.split().str[-1] 
#result['year'] = result['submit_date'].dt.year

In [14]:
result.dropna(how='any')
result.drop_duplicates()
result

,employer,job_title,base_salary,location,submit_date,start_date
0,COMTEC CONSULTANTS INC,BUSINESS SYSTEMS ANALYST,70000.0,KENNER LA,2014-11-07,2015-02-02
1,COMTEC CONSULTANTS INC,COMPUTER PROGRAMMER ANALYST,75000.0,KENNER LA,2014-10-24,2014-10-27
2,COMTEC CONSULTANTS INC,COMPUTER PROGRAMMER ANALYST,75000.0,KENNER LA,2014-11-26,2015-03-22
3,COMTEC CONSULTANTS INC,COMPUTER PROGRAMMER ANALYST,75000.0,KENNER LA,2014-12-19,2015-03-22
4,COMTEC CONSULTANTS INC,COMPUTER PROGRAMMER ANALYST,90000.0,KENNER LA,2014-11-26,2015-04-16
...,...,...,...,...,...,...
2419,GOLDMAN SACHS & CO LLC,VICE PRESIDENT,300000.0,NEW YORK NY,2021-03-03,2021-09-02
2420,GOLDMAN SACHS & CO LLC,VICE PRESIDENT,300000.0,NEW YORK NY,2021-03-15,2021-09-02
2421,GOLDMAN SACHS & CO LLC,VICE PRESIDENT,300000.0,NEW YORK NY,2021-03-19,2021-07-23
2422,GOLDMAN SACHS & CO LLC,VICE PRESIDENT,400000.0,NEW YORK NY,2021-02-23,2021-08-24


In [15]:
result.groupby('employer').count().sort_values(by='location',ascending=False).head(10)

,job_title,base_salary,location,submit_date,start_date
employer,,,,,
DELOITTE CONSULTING LLP,221693,221693,221693,221693,221693
INFOSYS LIMITED,127370,127370,127370,127370,127370
ERNST & YOUNG US LLP,92492,92540,92540,92540,92540
TATA CONSULTANCY SERVICES LIMITED,91453,91455,91455,91455,91455
COGNIZANT TECHNOLOGY SOLUTIONS US CORP,80789,80789,80789,80789,80789
WIPRO LIMITED,79089,79091,79091,79091,79091
LARSEN & TOUBRO INFOTECH LIMITED,69260,69260,69260,69260,69260
IBM INDIA PRIVATE LIMITED,67201,67207,67207,67207,67207
TECH MAHINDRA (AMERICAS)INC,52714,52716,52716,52716,52716


In [57]:
result[result.employer == 'SALESFORCECOM INC'].base_salary.mean()

128723.99064940403

In [ ]:
#result = pd.read_csv("data/1-10.csv")

In [ ]:
#sns.scatterplot(data= result.groupby('year').count().reset_index(),x='year',y='employer')

## Grouping similar companies

In [62]:
import numpy as np
from sklearn.cluster import AffinityPropagation
import distance

In [67]:
result.employer.unique()

2630

In [91]:
list1 = result.employer.unique()[:50]

In [92]:
list1

array(['MCAFEE LLC', '\n\n', '\n\n\n\n\n', 'PALAYEKAR COMPANIES INC',
       'TRIZETTO CORPORATION A COGNIZANT COMPANY',
       'EGROVE SYSTEMS CORPORATION', 'THE BROAD INSTITUTE INC',
       'FLEXTON INC', 'AKT LLC', 'MISSISSIPPI STATE UNIVERSITY',
       'VIGNA INC', 'AMITI CONSULTING INC', 'TEKSTROM INC',
       'CLOUD HUB IT SOLUTIONS INC',
       'THE DEPOSITORY TRUST AND CLEARING CORPORATION',
       'ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA UNIVERSITY',
       'ALBERT EINSTEIN COLLEGE OF MEDICINE',
       'IBS SOFTWARE SERVICES AMERICAS INC',
       'SPI MANAGEMENT SERVICES LLC', 'CLEVELAND CLINIC FOUNDATION',
       'UNIVERSITY OF MASSACHUSETTS AMHERST',
       'GLOBAL DATA MANAGEMENT INC', 'XENONINFOTEK INC',
       'SYNERGY TECHNOLOGIES LLC', 'RSM US LLP',
       'SEMICONDUCTOR COMPONENTS INDUSTRIES LLC',
       'EPEOPLE TECHNOLOGIES INC', 'CIRA INFOTECH INC',
       'NEW YORK LIFE INSURANCE COMPANY',
       'NORTHWESTERN MUTUAL LIFE INSURANCE COMPANY',
       'TECHNOVI

In [93]:
words = list1 #Replace this line
words = np.asarray(words) #So that indexing with a list will work
lev_similarity = -1*np.array([[distance.levenshtein(w1,w2) for w1 in words] for w2 in words])

affprop = AffinityPropagation(affinity="precomputed", damping=0.5)
affprop.fit(lev_similarity)
for cluster_id in np.unique(affprop.labels_):
    exemplar = words[affprop.cluster_centers_indices_[cluster_id]]
    cluster = np.unique(words[np.nonzero(affprop.labels_==cluster_id)])
    cluster_str = ", ".join(cluster)
    print(" - *%s:* %s" % (exemplar, cluster_str))

 - *TRIZETTO CORPORATION A COGNIZANT COMPANY:* TRIZETTO CORPORATION A COGNIZANT COMPANY
 - *AKT LLC:* 

, 




, AKT LLC, ALIXPARTNERS LLP, APTUDE INC, COHESITY INC, FLEXTON INC, FMR LLC, MAJESCO, MAJESCOMASTEK, MCAFEE LLC, MERKLE INC, RSM US LLP, TEKSTROM INC, VIGNA INC
 - *THE DEPOSITORY TRUST AND CLEARING CORPORATION:* THE DEPOSITORY TRUST AND CLEARING CORPORATION
 - *ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA UNIVERSITY:* ALBERT EINSTEIN COLLEGE OF MEDICINE, ALBERT EINSTEIN COLLEGE OF MEDICINE OF YESHIVA UNIVERSITY
 - *IBS SOFTWARE SERVICES AMERICAS INC:* IBS SOFTWARE SERVICES AMERICAS INC
 - *UNIVERSITY OF MASSACHUSETTS AMHERST:* UNIVERSITY OF MASSACHUSETTS AMHERST
 - *SYNERGY TECHNOLOGIES LLC:* EPEOPLE TECHNOLOGIES INC, SPI MANAGEMENT SERVICES LLC, SYNERGY TECHNOLOGIES LLC
 - *SEMICONDUCTOR COMPONENTS INDUSTRIES LLC:* SEMICONDUCTOR COMPONENTS INDUSTRIES LLC
 - *CIRA INFOTECH INC:* CAMBAY CONSULTING LLC, CIRA INFOTECH INC, GLOBAL DATA MANAGEMENT INC, NAM INFO INC, RINGCENTRAL 

In [99]:
sum =0
for year in range(2012,2022):
    df = load_data('INFOSYS LIMITED',year)
    print(len(df))
    sum = sum + len(df)
print(sum)

0
0
5125
31290
26126
19272
10935
21553
8146
4971
127418


In [100]:
sum =0
for year in range(2012,2022):
    df = load_data('DELOITTE CONSULTING LLP',year)
    print(len(df))
    sum = sum + len(df)
print(sum)

0
0
1215
7778
7250
8869
15388
8595
6040
50006
105141


In [104]:
result[result.employer=='DELOITTE CONSULTING LLP'].drop_duplicates()

,employer,job_title,base_salary,location,submit_date,start_date
0,DELOITTE CONSULTING LLP,ANALYST,72000.0,DENVER CO,2014-12-05,2015-01-11
1,DELOITTE CONSULTING LLP,ANALYST,75046.0,SAN FRANCISCO CA,2014-12-24,2015-01-12
2,DELOITTE CONSULTING LLP,ASSOCIATE 4,57555.0,INDIANAPOLIS IN,2014-12-17,2015-01-25
3,DELOITTE CONSULTING LLP,ASSOCIATE 4,58136.0,HARRISBURG PA,2014-12-15,2014-12-16
4,DELOITTE CONSULTING LLP,ASSOCIATE 5,61000.0,ORLANDO FL,2014-10-23,2014-10-24
...,...,...,...,...,...,...
4751,DELOITTE CONSULTING LLP,STUDIO STAFF,86800.0,GREENSBORO NC,2021-03-15,2021-08-15
4752,DELOITTE CONSULTING LLP,STUDIO STAFF,105000.0,NATICK MA,2021-05-05,2021-10-01
4753,DELOITTE CONSULTING LLP,SYSTEMS ENGINEERING SOFTWARE AUTOMATION TESTIN...,170000.0,ATLANTA GA,2021-05-28,2021-06-11
5260,DELOITTE CONSULTING LLP,VICE PRESIDENT,192130.0,BOSTON MA,2021-08-30,2021-09-03


In [106]:
result.drop_duplicates()

,employer,job_title,base_salary,location,submit_date,start_date
0,MCAFEE LLC,APPLICATION DEVELOPER,105500.0,SANTA CLARA CA,2017-11-30,2017-12-06
1,MCAFEE LLC,APPLICATION DEVELOPER,107806.0,PLANO TX,2017-08-09,2018-02-05
2,MCAFEE LLC,DATA ANALYST,78957.0,PLANO TX,2017-03-10,2017-09-07
3,MCAFEE LLC,DATA ANALYST,90776.0,SANTA CLARA CA,2017-03-09,2017-09-08
4,MCAFEE LLC,DATA SCIENTIST,123071.0,SANTA CLARA CA,2017-03-20,2017-09-09
...,...,...,...,...,...,...
40752,WIPRO LIMITED,VLSI LEAD,91021.0,FOLSOM CA,2021-02-11,2021-02-11
40754,WIPRO LIMITED,VLSI LEAD,92498.0,EDISON NJ,2021-04-01,2021-04-01
40759,WIPRO LIMITED,VLSI LEAD,106350.0,AUSTIN TX,2021-06-14,2021-06-14
40773,WIPRO LIMITED,VLSI LEAD,117291.0,SAN JOSE CA,2021-02-08,2021-02-09
